In [13]:
from tqdm import tqdm
from pathlib import Path
import numpy as np
import re

from utils.dataset import data_load,source_data_clear

### API

In [8]:
model_label_map = {
    0: 10,  # 鞋子
    1: 8,  # 包包
    2: 4,  # 上装
    3: 5,  # 裤子
    4: 6,  # 裙子
    5: 7,  # 连体装
}

In [9]:
id2chi = {
    10: '鞋子',
    8: '包包',
    4: '上装',
    5: '裤子',  # 裤子
    6: '半身裙',  # 裙子
    7: '连体装',  # 连体装
}

In [10]:
logic_id2model_id = {
    10: 0,
    8: 1,
    4: 2,
    5: 3,
    6: 4,  
    7: 5
}

### PIPELINE

In [21]:
img_origin = Path('/home/chenyi/workspace/dataset/data/data_fashionpedia/')

In [22]:
%%time
df = data_load(img_origin, depth=2)

CPU times: user 273 ms, sys: 15.8 ms, total: 289 ms
Wall time: 379 ms


In [23]:
%%time
df = source_data_clear(df, thres=400)

检查图片是否损坏...


100%|███████████████████████████████████████████████| 48823/48823 [00:52<00:00, 929.20it/s]


完成图片损坏检查.
完成图片分辨率检查.


100%|█████████████████████████████████████████████| 48654/48654 [00:01<00:00, 26635.71it/s]


完成图片去重,去重后图片数量是48654.
CPU times: user 1.93 s, sys: 214 ms, total: 2.15 s
Wall time: 55.3 s


In [20]:
df.to_csv('./data/drop_dup.csv', index=False)

In [8]:
df['stem_id'] = df['path'].apply(lambda x: Path(x).stem)
df['id'] = df['stem_id'].apply(lambda x: int(re.findall(r'_([0-9]{1,2})', x)[0]))
df['stem'] = df['stem_id'].apply(lambda x: re.findall(r'(.*?)_[0-9]{1,2}', x)[0])
df['catg_chi'] = df['id'].map(id2chi)

In [9]:
counter_list = [0]*6
ids_list = []
for id_ in tqdm(df['id'].values):
    id_model = logic_id2model_id[int(id_)]
    ids_list.append(counter_list[id_model])
    counter_list[id_model] += 1

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 75445/75445 [00:00<00:00, 3375208.15it/s]


In [10]:
folder_num = 100
root_target = Path('/nas/chenyi/datasets_cls/deploy_data/20220803/category/')
df['catg_id'] = ids_list
df['f_target'] = df['catg_id'].apply(lambda x: x//folder_num)
df['img_tp'] = df.apply(lambda row: '{0}/{1}/{2}/{3}_{4}.jpg'.format(root_target, row['catg_chi'], row['f_target'], row['stem'], row['catg_chi']), axis=1)

In [11]:
# df.to_csv('/nas/chenyi/datasets_cls/deploy_data/20220803/category/drop_duplicate_df.csv', index=False)

### 

In [203]:
from pandarallel import pandarallel
pandarallel.initialize(nb_workers=20)

INFO: Pandarallel will run on 20 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [204]:
def img_df_save(row):
    sp = Path(row['path'])
    tp = Path(row['img_tp'])
    if tp.is_file(): return
    if not tp.parent.is_dir():
        tp.parent.mkdir(parents=True, exist_ok=True)
    if sp.suffix == 'jpg':
        shutil.copyfile(sp, tp)
    else:
        cv2.imwrite(str(tp), cv2.imread(str(sp)))

In [205]:
%%time
df1 = df
_ = df1.parallel_apply(lambda row: img_df_save(row), axis=1)

CPU times: user 89.1 ms, sys: 179 ms, total: 268 ms
Wall time: 3min 6s


In [ ]:
import shutil
import cv2
for ind, row in tqdm(df.iterrows()):
    sp = Path(row['path'])
    tp = Path(row['img_tp'])
    if tp.is_file(): continue
    if not tp.parent.is_dir():
        tp.parent.mkdir(parents=True, exist_ok=True)
    if sp.suffix == 'jpg':
        shutil.copyfile(sp, tp)
    else:
        cv2.imwrite(str(tp), cv2.imread(str(sp)))
    # break